# A1 - OpenJij core interface入門

このチュートリアルではOpenJijのcore interfaceの使い方を説明し、簡単な計算のデモンストレーションを行います。

core interfaceは前回までのチュートリアルよりも下部のレイヤーのAPIとなるため、対象読者としては前回までのOpenJijチュートリアルを一通り終えて、イジングモデルやモンテカルロ法などの用語を知っている方を想定しています。
また、C++インターフェイスについても同時に紹介するため、ある程度のC++の知識があると望ましいです。

## OpenJij core interface について

前回までのチュートリアルではOpenJijを用いた様々な問題の解き方やベンチマークの取り方などを紹介してきましたが、OpenJijは最下層の部分は統計物理学の数値計算手法であるマルコフ連鎖モンテカルロ法をベースにC++を用いて実装されており、今まで触れてきたpythonモジュールはこのC++インターフェースを直接ラップしたpythonライブラリである**cxxjij**を呼び出す形で実装されています。図にすると次のような包含関係があります。

[TODO: 図形]

core interfaceを用いることでOpenJij上の全ての機能を使用することができるため、最適化問題のみならず、統計物理学の数値計算ツールとして研究用途で使用することもできます。
また、C++インターフェースを用いることでより高速な演算を行うことができます。
本チュートリアルではpythonインターフェースのcxxjijと、C++インターフェースの両方を紹介します。
インストールにはpipを使用します。

In [93]:
!pip install openjij

Defaulting to user installation because normal site-packages is not writeable


## 問題を投げてみる

チュートリアルとしてまずは変数のサイズが$N=5$の古典スピン ($\sigma = \pm 1$)イジング問題をアニーリングで解いてみましょう。ハミルトニアンは以下のようになります。
\begin{align}
H &= \sum_{i<j}J_{ij}\sigma_i \sigma_j + \sum_{i=1}^{N}h_i \sigma_i \\
\sigma_i &= \pm 1 (i=1 \cdots N)
\end{align}

縦磁場と相互作用が
$h_i = -1 \ \mathrm{for\ } \forall i,\ J_{ij} = -1 \mathrm{for\ } \forall i,j$
だと各スピンは1の値をとった方がエネルギーが低くなるため、$\{\sigma_i\} = \{1,1,1,1,1\}$が最適解となります。この問題を解いてみましょう。
pythonコードを用いた一通りの流れは次のようになります。

In [102]:
# core interfaceではopenjijの代わりにcxxjijをインポートします。
import cxxjij as cj

# まず相互作用行列を作成してあげます。Graphモジュールを使います。
import cxxjij.graph as G

# 問題サイズN=5の密結合グラフ(Dense)を定義します。
N = 5
J = G.Dense(N)
# 相互作用を設定してあげます。
for i in range(N):
    for j in range(N):
        #J[i,i]以外に-1を入力
        J[i,j] = 0 if i == j else -1.0

# 縦磁場を設定してあげます。
for i in range(N):
    J[i] = -1

# 続いてGraphから計算を行うためのSystemを作成します。
import cxxjij.system as S

# 今回は通常の古典モンテカルロ計算のシステムを使用します。
# (Eigenは)
system = S.make_classical_ising_Eigen(J.gen_spin(), J)
# アニーリングスケジュールを設定します。Utilityモジュールを使用します。
import cxxjij.utility as U

schedule = U.make_classical_schedule_list(0.1, 100, 10, 10)
# 実際にアニーリングを走らせます。Algorithmモジュールを使用します。
# モンテカルロステップの更新方法として単純なSingleSpinFlipを用います。
import cxxjij.algorithm as A

A.Algorithm_SingleSpinFlip_run(system, schedule)
# 結果を取得します。Resultモジュールにあるget_solutionを用います。
import cxxjij.result as R

print(R.get_solution(system))

[1, 1, 1, 1, 1]


出てきた答えが$[1,1,1,1,1]$になったのが確認できます。
低レイヤーのAPIのため、設定する項目は多いですがその分柔軟な設定が可能となります。

## モジュール一覧

コード例に出てきたように、OpenJij core interfaceでは主に次のモジュール群からなります。この章では簡単な紹介にとどめておき、詳細な解説は次章以降で行います。

### `Graph`

問題のグラフ構造 (相互作用係数$J_{ij}$)を定義するためのモジュールです。密結合や疎結合に対応したクラスを持っており、タイプに応じて最適化された実装がなされています。

### `System`

モンテカルロ等の計算を行うためのシステムを定義するためのモジュールです。使用する手法に応じて様々なクラス (古典イジングシステム、横磁場イジングシステム等)が用意されています。

### `Updater`

どのような手法で`System`